# Exercise_1

After execution of all cells, choose a scenario with click on the respective button in the GUI. Then click on the "Start"-button to run the simulation. To avoid unexpected behavior, it's recommended to wait until the scenario is finished (i.e. the seconds are not counted further) or to close the GUI window and execute the "main"-cell again.

In [ ]:
import tkinter as tk
import numpy as np
import math
import time
import seaborn as sns
import random

In [ ]:
#list for measurement for RiMEA 4 scenario
measurement = []

In [ ]:
class Person():
    '''
    pos_x, pos_y: position of pedestrian
    speed: in cells/timestep. if timestep=1s, cellsize=0.25m and desiredSpeed=1.6m/s -> speed needs to be 1.6/0.25=6.4 cells/timestep
    move(): see below
    '''
    def __init__(self, position_x, position_y, speed=1):
        self.pos_x = position_x
        self.pos_y = position_y
        self.speed = speed 
        self.leap = 0 # >0 bonus for next step, <0 penalty for next step
        
    def move(self, grid, scenario):
        '''
        Equals an update for the position depending on speed, interaction of pedestrians and cost field
        looks for minimal cost in all directions within the feasible area
        Effectivley updates position by passing position to grid
        '''
        self.pedestrianInteraction(scenario)#updates interactionField
        costField = np.array(scenario.costField) + np.array(self.interactionField)
        ## for debugging of costField, interactionField .. 
        #for (x,y),v in np.ndenumerate(np.array(self.interactionField)):
        #    if v > 0:
        #        print('('+str(x),str(y)+')'+': '+str(v))
        #        print('cost single'+': '+str(scenario.costField[x][y]))
        #        print('cost combined'+': '+str(costField[x][y]))
        ##
        (x_opt, y_opt) = (self.pos_x, self.pos_y)
        cost_min = costField[round(self.pos_x)][round(self.pos_y)]
        
        #assuming constant timesteps of 1s
        pos_dist = self.leap + self.speed #possible distance for the pedestrian to move in this timestep
        if round(pos_dist) <= 0:#pedestrian needs to wait in order to move with correct speed
            self.leap += self.speed
            return
            
        while round(pos_dist) > 0:#if move at this timestep 
            #steps only +-1
            dxplus = self.pos_x + 1
            dxminus = self.pos_x - 1
            dyplus = self.pos_y + 1
            dyminus = self.pos_y - 1

            dxx = (self.pos_x, dxplus, dxminus)
            dyy = (self.pos_y, dyplus, dyminus)

            cell_move = (cost_min, self.pos_x, self.pos_y)             

            flag_diag = False #indicates wether the pedestrian moves diagonal or not
            
            for x in dxx:
                for y in dyy:
                    if scenario.isInDomain(x,y):
                        if (x,y) == (self.pos_x, self.pos_y):
                            continue#no need to check own position again
                        else:
                            if scenario.vanish and grid.grid[x][y] == 'T':#allows vanishing persons e.g. through doors
                                grid.grid[round(self.pos_x)][round(self.pos_y)]='E'
                                scenario.persons.remove(self)
                                return
                            if grid.grid[x][y] == 'E':#neighbored cell is empty
                                cell_dist = costField[x][y]
                                if  cell_dist < cell_move[0]:
                                    cell_move = (cell_dist,x,y)
                                    if ((self.pos_x - x)+(self.pos_y - y))%2 == 0:#diagonal movement yes/no
                                        flag_diag = True
                                    else:
                                        flag_diag = False
            
            #updates grid for drawing and own position
            grid.grid[round(self.pos_x)][round(self.pos_y)]='E'
            grid.grid[round(cell_move[1])][round(cell_move[2])]='P'
            self.pos_x = cell_move[1]
            self.pos_y = cell_move[2]
            
            #calculate new (remaining) possible distance depending on  (striaght/diag) and update self leap
            if flag_diag == True:
                pos_dist -= math.sqrt(2)
                self.leap = pos_dist
            else:
                pos_dist -= 1
                self.leap = pos_dist        
        
        
    def pedestrianInteraction(self, scenario):#TODO: needs isInDomain() check
        self.interactionField = [[0.0 for y in range(scenario.columns)] for x in range(scenario.rows)]
        for p in scenario.persons:
            if p == self:
                continue#this is to avoid penalty with yourself. I assume you like yourself :)
            else:
                x = round(p.pos_x)
                y = round(p.pos_y)
                dxx = [x]
                dyy = [y]
                for r in range(scenario.rmax)[1:]:#rmax = 3 -> (0,)1,2
                    dxx.append(x+r)
                    dxx.append(x-r)
                    dyy.append(y+r)
                    dyy.append(y-r)
                for xx in dxx:
                    for yy in dyy:
                        if scenario.isInDomain(xx,yy):
                            if (xx - x)**2 + (yy - y)**2 >= scenario.rmax**2:#cell not within cirlce around pedestrian
                                continue
                            r = math.sqrt((x - xx)**2 + (y - yy)**2)#eucliddistance between p.pos_xy and xxyy
                            self.interactionField[xx][yy] += math.exp(1/(r**2 - scenario.rmax**2))

In [ ]:
class Scenario():
    '''
    creates persons, targets, obstacles and grid size depending on the scenario
    rmax: maximum radius of pedestrian interaction. Depends on cellsize -> adjust for each scenario
    '''
    def __init__(self):
        self.rows = 25
        self.columns = 25
        self.persons = []
        self.targets = []
        self.obstacles = []
        self.grid = Grid(self)
        self.rmax = 3
        self.time_limit = 20
        self.vanish = False #Set to true for scenarios where the target is a door
        self.fastMarching = False # bottleneck and chickentest both once without Dijkstra/fastMarching and once with
        self.line_distance = 20
        
    def setTo1(self):#one person straight
        self.rows = 50
        self.columns = 50
        self.persons = [Person(5, 25)]
        self.targets = [(25, 25)]
        self.obstacles = []
        self.rmax = 3
        self.time_limit = 25
        self.vanish = False
        self.fastMarching = False
        self.setupCostField()
        self.grid = Grid(self)
        self.line_distance = 20
        
    def setTo2(self):#five persons in a circle
        self.rows = 50
        self.columns = 50
        self.persons = [Person(5, 25), Person(25, 45), Person(25, 5), Person(41, 37), Person(41, 13)]
        self.targets = [(25, 25)]
        self.obstacles = []
        self.rmax = 3
        self.time_limit = 25
        self.vanish = False
        self.fastMarching = False
        self.setupCostField()
        self.grid = Grid(self)
        self.line_distance = 20
        
    def setTo3(self):#bottleneck
        self.rows = 54
        self.columns = 54
        p = []
        for i in range(10):
            for j in range(20):
                if j%4 is not 0:
                    p.append((Person(i+2,j+2,3)))
        self.persons = p
        self.targets = [(52, 11), (52, 12)]
        list = []
        for i in range(10):
            #left room
            list.extend([(1,11-i),(1,12+i),(2+i,1),(2+i,22),(12+i,1),(12+i,22),(22,11-i),(22,12+i)])
            #bottleneck
            list.append((23+i,10))
            list.append((23+i,13))
            #right room
            list.extend([(31,11-i),(31,12+i),(32+i,1),(32+i,22),(42+i,1),(42+i,22),(52,11-i),(52,12+i)])
        list.remove((32, 10))
        list.remove((32, 13))
        list.remove((22,11))
        list.remove((22,12))
        list.remove((31,11))
        list.remove((31,12))
        list.remove((52,11))
        list.remove((52,12))
        #corners
        list.extend([(1,1), (1,22), (22,1), (22,22), (31,1), (31,22), (52,1), (52,22)])
        self.obstacles = list
        self.rmax = 2
        self.time_limit = 90
        self.vanish = True
        self.fastMarching = False
        self.setupCostField()
        self.grid = Grid(self)
        self.line_distance = 20
        
    def setTo3wD(self):#bottleneck with Dijkstra
        self.rows = 54
        self.columns = 54
        p = []
        for i in range(10):
            for j in range(20):
                if j%4 is not 0:
                    p.append((Person(i+2,j+2,3)))
        self.persons = p
        self.targets = [(52, 11), (52, 12)]
        list = []
        for i in range(10):
            #left room
            list.extend([(1,11-i),(1,12+i),(2+i,1),(2+i,22),(12+i,1),(12+i,22),(22,11-i),(22,12+i)])
            #bottleneck
            list.append((23+i,10))
            list.append((23+i,13))
            #right room
            list.extend([(31,11-i),(31,12+i),(32+i,1),(32+i,22),(42+i,1),(42+i,22),(52,11-i),(52,12+i)])
        list.remove((32, 10))
        list.remove((32, 13))
        list.remove((22,11))
        list.remove((22,12))
        list.remove((31,11))
        list.remove((31,12))
        list.remove((52,11))
        list.remove((52,12))
        #corners
        list.extend([(1,1), (1,22), (22,1), (22,22), (31,1), (31,22), (52,1), (52,22)])
        self.obstacles = list
        self.rmax = 2
        self.time_limit = 90
        self.vanish = True
        self.fastMarching = True
        self.setupCostField()
        self.grid = Grid(self)
        self.line_distance = 20
        
    def setTo4(self):#chickentest
        self.rows = 25
        self.columns = 25
        self.persons = [Person(6, 12)]
        self.targets = [(18, 12)]
        self.obstacles = [(12,12),(12,13),(12,11),(12,14),(12,10),(12,15),(12,9),(11,15),(10,15),(11,9),(10,9)]
        self.rmax = 3
        self.time_limit = 20
        self.vanish = False
        self.fastMarching = False
        self.setupCostField()
        self.grid = Grid(self)
        self.line_distance = 20
        
    def setTo4wD(self):#chickentest with Dijkstra
        self.rows = 25
        self.columns = 25
        self.persons = [Person(6, 12)]
        self.targets = [(18, 12)]
        self.obstacles = [(12,12),(12,13),(12,11),(12,14),(12,10),(12,15),(12,9),(11,15),(10,15),(11,9),(10,9)]
        self.rmax = 3
        self.time_limit = 20
        self.vanish = False
        self.fastMarching = True
        self.setupCostField()
        self.grid = Grid(self)
        self.line_distance = 20
        
    ###Debugging scenario
    #def setTo5(self):#added 5th scenario to test hypothesis
    #    self.rows = 50
    #    self.columns = 50
    #    self.persons = [Person(5, 45), Person(10, 47), Person(3, 42), Person(18, 43), Person(46, 44), Person(38,41), Person(45,44), Person(45,42)]
    #    self.targets = [(25, 0)]
    #    self.obstacles = []
    #    self.rmax = 3
    #    self.time_limit = 25
    #    self.vanish = False
    #    self.fastMarching = False
    #    self.setupCostField()
    #    self.grid = Grid(self)
        
    def rimea1(self):#RiMEA Scenario 1: Straight Line Corridor
        self.rows = 50
        self.columns = 50
        self.persons = [Person(8, 25-20, 1.33)]
        self.targets = [(48, 25-20)]
        list = [(49,25-20),(49,24-20)]
        for i in range(8,50):
            list.append((i,26-20))
            list.append((i,23-20))
        self.obstacles = list
        self.rmax = 3
        self.time_limit = 35
        self.vanish = False
        self.fastmarching = True
        self.setupCostField()
        self.grid = Grid(self)
        self.line_distance = 20
        
        #Rimea 4 : 0.5p/sqm density
                
    '''def rimea4(self):#RiMEA Scenario 4: Measurement of the Fundamental Diagram
        self.rows = 50
        self.columns = 50
        #plist = [Person(50, 25, 1.2), Person(50, 27, 1.2)]
        plist = []
        #l = [(random.randrange(2, 30), random.randrange(21, 31)) for k in range(200)]
        #for e in l:
            #plist.append(Person(e[0],e[1],1.2))
        for i in range(0,50):
            if (i % 2) == 0:
                for j in range(21,31):
                    #if (j % 2) == 0:
                        plist.append(Person(i,j,1.2))
        #self.targets = [(99,11),(99,12),(99,13),(99,14),(99,15),(99,16)]
        self.persons = plist
        tlist = []
        olist = []
        for i in range(0,50):
            olist.append((i,20))
            olist.append((i,31))
        for i in range(21,31):
            tlist.append((49,i))
        self.rmax = 2
        self.obstacles = olist
        self.targets = tlist
        self.fastmarching = True
        self.setupCostField()
        self.grid = Grid(self)
        self.line_distance = 10
        self.vanish = True
        self.time_limit = 100'''
        
        
        #Rimea 4 : 1p/sqm density
    def rimea4(self):#RiMEA Scenario 4: Measurement of the Fundamental Diagram
        self.rows = 50
        self.columns = 50
        #plist = [Person(50, 25, 1.2), Person(50, 27, 1.2)]
        plist = []
        #l = [(random.randrange(2, 80), random.randrange(13, 38)) for k in range(2)]
        #for e in l:
            #plist.append(Person(e[0],e[1],1.2))
        for i in range(1,49):
            for j in range(21,31):
                plist.append(Person(i,j,1.2))
        #self.targets = [(99,11),(99,12),(99,13),(99,14),(99,15),(99,16)]
        self.persons = plist
        tlist = []
        olist = []
        for i in range(0,50):
            olist.append((i,20))
            olist.append((i,31))
        for i in range(21,31):
            tlist.append((49,i))
        self.rmax = 1
        self.obstacles = olist
        self.targets = tlist
        self.fastmarching = True
        self.setupCostField()
        self.grid = Grid(self)
        self.line_distance = 10
        self.vanish = True
        self.time_limit = 100
        
        #Rimea 4 : 2p/sqm density
    '''def rimea4(self):#RiMEA Scenario 4: Measurement of the Fundamental Diagram
        self.rows = 50
        self.columns = 50
        #plist = [Person(50, 25, 1.2), Person(50, 27, 2.4)]
        plist = []
        #l = [(random.randrange(2, 80), random.randrange(13, 38)) for k in range(2)]
        #for e in l:
            #plist.append(Person(e[0],e[1],1.2))
        for i in range(1,49):
            if (i % 2) == 0:
                for j in range(21,41):
                    plist.append(Person(i,j,2.4))
        #self.targets = [(99,11),(99,12),(99,13),(99,14),(99,15),(99,16)]
        self.persons = plist
        tlist = []
        olist = []
        for i in range(0,50):
            olist.append((i,20))
            olist.append((i,41))
        for i in range(21,41):
            tlist.append((49,i))
        self.rmax = 2
        self.obstacles = olist
        self.targets = tlist
        self.fastmarching = True
        self.setupCostField()
        self.grid = Grid(self)
        self.line_distance = 10
        self.vanish = True
        self.time_limit = 100'''
        
        #Rimea 4 : 6p/sqm density (not working)
    '''def rimea4(self):#RiMEA Scenario 4: Measurement of the Fundamental Diagram
        self.rows = 50
        self.columns = 100
        #self.persons = [Person(1, 25, 1.2), Person(3,25,1.2), ]
        listp = []
        l = [(random.randrange(2, 80), random.randrange(13, 38)) for k in range(2)]
        for e in l:
            listp.append(Person(e[0],e[1],1.2))
        #for i in range(1,80):
            #for j in range(12,39):
                #listp.append(Person(i,j,1.2))
        #self.targets = [(99,11),(99,12),(99,13),(99,14),(99,15),(99,16)]
        self.persons = listp
        listb = []
        list = []
        for i in range(0,99):
            list.append((i,10))
            list.append((i,40))
        for i in range(11,40):
            listb.append((99,i))
        self.rmax = 9
        self.obstacles = list
        self.targets = listb
        self.fastmarching = True
        self.setupCostField()
        self.grid = Grid(self)
        self.line_distance = 10
        self.vanish = True
        self.time_limit = 100'''
    
    def rimea6(self):#RiMEA Scenario 6: Movement Around Corridor
        self.rows = 50
        self.columns = 50
        self.persons = [Person(10, 24),Person(9, 24),Person(8, 24),Person(7, 24),Person(6, 24),Person(5, 24),Person(4, 24),Person(11, 24),Person(12, 24),Person(13, 24),
        Person(10, 25),Person(9, 25),Person(8, 25),Person(7, 25),Person(6, 25),Person(5, 25),Person(4, 25),Person(13, 25),Person(12, 25),Person(11, 25)]
        self.targets = [(20, 13),(19,13)]
        list = [(49,25),(49,24)]
        for i in range(0,19):
            list.append((i,23))
        for k in range(0,22):
            list.append((k,26))
        for z in range(13,23):
            list.append((18,z))
        for j in range(13,27):
            list.append((21,j))
        self.obstacles = list
        self.rmax = 10
        self.time_limit = 50
        self.vanish = True
        self.fastMarching = True
        self.setupCostField()
        self.grid = Grid(self)
        self.line_distance = 20
    
    def rimea7(self):#RiMEA Scenario 7: average speed
        self.rows = 50
        self.columns = 50
        speed_list=[]
        for i in range(2,13):
            for k in range(2,8):
                speed=np.random.normal(1.5,0.25)
                p=Person(i,k,speed)
                self.persons.append(p)
                speed_list.append(speed)
        print("SPEED-LIST:",speed_list)
        avg_speed=np.mean(speed_list)
        minimum=np.min(speed_list)
        maximum=np.max(speed_list)
        print("minimum speed:", minimum,"maximum speed: ", maximum,
        "average speed: ", avg_speed)
        sns.kdeplot(speed_list)
        self.targets = [(49, 5)]
        list = []
        for z in range(0,50):
            list.append((z,1))
        for j in range(0,50):
            list.append((j,9))
        self.obstacles = list
        self.rmax = 10
        self.time_limit = 50
        self.vanish = True
        self.fastMarching = True
        self.setupCostField()
        self.grid = Grid(self)
        self.line_distance = 20
    
    #TODO: add RiMEA tests only 6 missing
        
    def setupCostField(self):
        #cost scaled by max possible cost/distance
        if self.fastMarching == False:#rudimentary obstacle avoidance only
            self.costField = [[1.0 for y in range(self.columns)] for x in range(self.rows)]
            max_dist = math.sqrt(self.rows**2 + self.columns**2)
            for x in range(self.rows):
                for y in range(self.columns):
                    if (x,y) in self.targets:
                        self.costField[x][y] = 0
                    elif (x,y) in self.obstacles:
                        self.costField[x][y] = math.exp(10)
                    else:#computes cost/distance for nearest target
                        dist = max_dist
                        for t in self.targets:
                            min_dist = math.sqrt((x - t[0])**2 + (y - t[1])**2)
                            if min_dist < dist:
                                dist = min_dist
                        cost = dist#/max_dist # scaling by max_dist can cause pedestrians to stop before the target
                        self.costField[x][y] = cost
        else:#dijkstra or fastmarching
            self.Dijkstra()
            #self.fastMarching()
            
    def Dijkstra(self):
        #initialisation
        self.costField = [[np.inf for y in range(self.columns)] for x in range(self.rows)]
        Q = []
        for x in range(self.rows):
            for y in range(self.columns):
                if (x,y) in self.targets:
                    self.costField[x][y] = 0
                Q.append((x,y))
        #loop
        while len(Q) > 0:
            #find minimal value in Q
            x_min = Q[0][0]
            y_min = Q[0][1]
            for (x,y) in Q:
                if self.costField[x][y] < self.costField[x_min][y_min]:
                    (x_min, y_min) = (x,y)
            Q.remove((x_min,y_min))
            #print('Remaining values to calculate: '+str(len(Q)))
            #neighbors of x_min,y_min
            neighbors = [(x_min+1,y_min),(x_min,y_min+1),(x_min-1,y_min),(x_min,y_min-1),(x_min+1,y_min+1),(x_min+1,y_min-1),(x_min-1,y_min-1),(x_min-1,y_min+1)]
            for (nx,ny) in neighbors:
                if self.isInDomain(nx,ny):
                    #update distance for neighbors
                    if (nx,ny) in Q:
                        if (nx,ny) in self.obstacles:
                            self.costField[nx][ny] = math.exp(10)
                        else:
                            alternativ = self.costField[x_min][y_min] + math.sqrt((nx - x_min)**2 + (ny - y_min)**2)
                            if alternativ < self.costField[nx][ny]:
                                self.costField[nx][ny] = alternativ
                            
    def isInDomain(self,x,y):
        if 0 <= y < self.rows and 0 <= x < self.columns:
            return True
        else:
            return False
        
    def fastMarching(self):
        self.obstacleField = [[0.0 for y in range(self.columns)] for x in range(self.rows)]
        if self.fastMarching == False:#rudimentary obstacle avoidance only
            for x in range(self.rows):
                for y in range(self.columns):
                    if (x,y) in self.obstacles:
                        self.obstacleField[x][y] = math.exp(10)
        else:#Fast Marching algorithm
            # If there is time left, we can check on this again
            return
            accepted = []
            far = []
            considered = []
            #step 1
            for x in range(self.rows):
                for y in range(self.columns):
                    if (x,y) in self.obstacles:
                        self.obstacleField[x][y] = 0
                        accepted.append((x,y))
                    else:
                        self.obstacleField[x][y] = np.inf
                        far.append((x,y))
            #step 2
            for (x,y) in far:
                U = 1#some new value with update formula
                if U < self.obstacleField[x][y]:
                    self.obstacleField[x][y] = U
                    far.remove((x,y))
                    considered.append((x,y))   
            #steps 3-5
            while len(considered) > 0:
                #step 3
                v = np.inf
                for (x,y) in considered:
                    if self.obstacleField[x][y] < v:
                        v = self.obstacleField[x][y]
                        x_tilda = x
                        y_tilda = y
                considered.remove((x_tilda,y_tilda))
                accepted.append((x_tilda,y_tilda))
                #step 4
                neighbors = [(x_tilda+1,y_tilda),(x_tilda-1,y_tilda),(x_tilda,y_tilda+1),(x_tilda,y_tilda-1)]#diagonal neighbors too?
                for (x,y) in neighbors:
                    U = 1#update formula
                    if U < self.obstacleField[x][y]:
                        self.obstacleField[x][y] = U
                    #step 5
                    if (x,y) in far:
                        far.remove((x,y))
                        considered.append((x,y))
                
    def move(self):
        for p in self.persons:
            p.move(self.grid, self)

In [ ]:
class Grid():
    '''
    Basically only for visualization. 2d-array which stores markers for empty, pedestrian, target and obstacle.
    '''
    def __init__(self, scenario):
        self.scenario = scenario
        #self.grid = [['E' for y in range(scenario.columns)] for x in range(scenario.rows)]
        self.grid = [['E' for y in range(scenario.rows)] for x in range(scenario.columns)]
        for p in scenario.persons:
            self.grid[p.pos_x][p.pos_y] = 'P'
        for t in scenario.targets:
            self.grid[t[0]][t[1]] = 'T'
        for o in scenario.obstacles:
            self.grid[o[0]][o[1]] = 'O'
        
    def draw(self, canvas, line_distance):
        '''
        Draws the grid and the persons/targets/obstacles.
        '''
        
        # vertical lines
        for x in range(line_distance,canvas.winfo_width(),line_distance):
            canvas.create_line(x, 0, x, canvas.winfo_height(), fill="#476042")
        # horizontal lines
        for y in range(line_distance,canvas.winfo_height(),line_distance):
            canvas.create_line(0, y, canvas.winfo_width(), y, fill="#476042")
        # P, T, O or nothing in between the lines
        for (x, y), value in np.ndenumerate(self.grid):
            
            #measurement cells (2m x 2m) code block:
            if x == 25 and y == 25:# 0.5 & 1 p/sqm
                measurement.append(value)
            if x == 26 and y == 25:# 0.5 & 1 p/sqm
                measurement.append(value)
            
            #if x == 27 and y == 25:# 2p/sqm
                #measurement.append(value)
            #if x == 28 and y == 25:# 2p/sqm
                #measurement.append(value)
            
            if x == 25 and y == 26:# 0.5 & 1 p/sqm
                measurement.append(value)
            if x == 26 and y == 26:# 0.5 & 1 p/sqm
                measurement.append(value)
                
            #if x == 27 and y == 26:# 2p/sqm
                #measurement.append(value)
            #if x == 28 and y == 26:# 2p/sqm
                #measurement.append(value)
                
            #if x == 25 and y == 27:# 2p/sqm
                #measurement.append(value)
            #if x == 26 and y == 27:# 2p/sqm
                #measurement.append(value)
            
            #if x == 27 and y == 27:# 2p/sqm
                #measurement.append(value)
            #if x == 28 and y == 27:# 2p/sqm
                #measurement.append(value)
                
            #if x == 25 and y == 28:# 2p/sqm
                #measurement.append(value)
            #if x == 26 and y == 28:# 2p/sqm
                #measurement.append(value)

            #if x == 27 and y == 28:# 2p/sqm
                #measurement.append(value)
            #if x == 28 and y == 28:# 2p/sqm
                #measurement.append(value)
                
            #end of measurement code block
            
            if value =='P':
                #if (x == 25 and y == 25) or (x == 25 and y == 26) or (x == 26 and y == 25) or (x == 26 and y == 26):
                    #color = 'red'
                #else:
                    #color = 'green'   ## comment out block to make measurement area visible for scenario 4
                color = 'green'
            elif value =='T':
                color = 'red'
            elif value =='O':
                color = 'black'
            else:
                value =''
                color = 'white'
            canvas.create_text((x+0.5) * line_distance, (y+0.5) * line_distance, text=value, fill=color)
            # canvas.create_rectangle(..) as possible improvement for visualization

In [ ]:
def main():
    '''
    Setup of the interface and definition of the simulation loop.
    When a scenario is selected via a button, it appears next to the buttons in the canvas.
    When clicking on start button, the current scenario is simulated, i.e. for every timestep=1s each pedestrian moves and the drawing is renewed.
    '''
    window = tk.Tk()
    window.title('Exercise 1')
    
    scenario = Scenario()
    
    #(TODO:) DONE (more or less) make line_distance dependend on scenario columns/rows for nicer layout. Same for fontsize?
    line_distance = scenario.line_distance #should be passed to draw() in Grid
    canvas_width = line_distance * scenario.columns
    canvas_height = line_distance * scenario.rows
    canvas = tk.Canvas(master=window, width=canvas_width, height=canvas_height)
    canvas.pack(side='left')
    label = tk.Label(master=window, text='0 s', width='18')
    label.pack()
    
    btnScenario1 = tk.Button(master=window, text='1 Pedestrian', bg='#26f9ad', width='18', command=lambda: scenario.setTo1())
    btnScenario1.pack()#(side=RIGHT)
    btnScenario2 = tk.Button(master=window, text='Pedestrians in a circle', bg='#26f9ad', width='18', command=lambda: scenario.setTo2())
    btnScenario2.pack()#(side=RIGHT)
    btnScenario3 = tk.Button(master=window, text='Bottleneck', bg='#26f9ad', width='18', command=scenario.setTo3)
    btnScenario3.pack()#(side=RIGHT)
    btnScenario3wD = tk.Button(master=window, text='Bottleneck w/ Dijkstra', bg='#26f9ad', width='18', command=scenario.setTo3wD)
    btnScenario3wD.pack()#(side=RIGHT)
    btnScenario4 = tk.Button(master=window, text='Chickentest', bg='#26f9ad', width='18', command=scenario.setTo4)
    btnScenario4.pack()#(side=RIGHT)
    btnScenario4wD = tk.Button(master=window, text='Chickentest w/ Dijkstra', bg='#26f9ad', width='18', command=scenario.setTo4wD)
    btnScenario4wD.pack()#(side=RIGHT)
    #btnScenario5 = tk.Button(master=window, text='Scenario 5', bg='#26f9ad', width='18', command=scenario.setTo5)
    #btnScenario5.pack()#(side=RIGHT)
    btnScenarioR1 = tk.Button(master=window, text='RiMEA Scenario 1', bg='#26f9ad', width='18', command=scenario.rimea1)
    btnScenarioR1.pack()#(side=RIGHT)
    btnScenarioR4 = tk.Button(master=window, text='RiMEA Scenario 4', bg='#26f9ad', width='18', command=scenario.rimea4)
    btnScenarioR4.pack()#(side=RIGHT)
    btnScenarioR6 = tk.Button(master=window, text='RiMEA Scenario 6', bg='#26f9ad', width='18', command=scenario.rimea6)
    btnScenarioR6.pack()#(side=RIGHT)
    btnScenarioR7 = tk.Button(master=window, text='RiMEA Scenario 7', bg='#26f9ad', width='18', command=scenario.rimea7)
    btnScenarioR7.pack()#(side=RIGHT)
    
    
    def start(window, canvas, scenario, time_limit=scenario.time_limit):
        if scenario.rows or scenario.colummns <= 75:
            canvas_width = scenario.line_distance * scenario.columns
            canvas_height = scenario.line_distance * scenario.rows
        else:
            canvas_width = 40 * scenario.columns
            canvas_height = 40 * scenario.rows
        canvas.configure(width=canvas_width, height=canvas_height)
        canvas.delete('all')
        window.after(10, lambda: scenario.grid.draw(canvas, scenario.line_distance))
        time_limit = scenario.time_limit
        timesteps = 0
        while timesteps < time_limit:
            scenario.move()
            canvas.delete('all')
            time.sleep(1)#1 below 1000
            window.after(0, lambda: scenario.grid.draw(canvas, scenario.line_distance))
            window.update()
            timesteps += 1
            label.configure(text=str(timesteps)+' s')
            #print(measurement) #for rimea scen 4
    
    btnStart = tk.Button(master=window, text="Start", bg='lightblue', width='18', command=lambda: start(window, canvas, scenario))
    btnStart.pack()
    
    window.mainloop()

In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
#Function for Measurement of density and flow for scenario 4 - # speed is pedestrian still input speed, not real speed (TODO)

empty = 0
pedestrian = 0
for e in measurement:
    if e == 'E':
        empty = empty + 1
    else:
        pedestrian = pedestrian + 1
               
#avg_density = pedestrian/(((pedestrian + empty)/16))/4 # for 2p/sqm densities
#timestep = (pedestrian + empty)/16

avg_density = pedestrian/(((pedestrian + empty)/4))/4 #for 0.5p/sqm & 1p/sqm density
timestep = (pedestrian + empty)/4

#print(measurement)
#print(pedestrian)
#print(empty)
#print(avg_density)

In [ ]:
print("the average density at the measurement point after " + str(timestep) + "s is: " + str(avg_density) + " p/sqm")

In [ ]:
#result not accurate due to use of input speed instead of real speed
avg_flow = avg_density * 1.2
print("the average flow at the measurement point after " + str(timestep) + "s is: " + str(avg_flow) + " p/ms")